#### Relevant Libraries

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#### Add Relevant Libraries
#################################################
from datetime import datetime, time, date, timedelta
import pandas as pd
import numpy as np
import sys
import os
CWD = os.getcwd()
WD = CWD.split('Main')[0] + 'Main/'
DFNC = WD + 'Functions'
sys.path.insert(1, DFNC)

#### Add External Functions
#################################################
from Scripts.FN_Support import Drct, Periods


### Updated Patterns
- Below is an updated summery of patterns
- Several of these patterns are new from what I sent you last time
- All of them have a different column naming structure (more on that below...)

In [4]:
### Set the AGG Period Type
#######################################
AGG='5T'

### Load Pattern Summeries for ALL_CODES
#######################################
SUMMERY_FILE = f'{WD}Sources/aggs/{AGG}/Results/Pattern_Summeries/All.csv'
summeries = Drct.csv_from(SUMMERY_FILE)
ALL_CODES = list(summeries["pcode"].unique())
print(summeries[['pcode','filled']])


   pcode  filled
0   3BPD   17777
1   3BPU   18429
2   BFLD   13953
3   BFLU   14734
4    BOD   42491
5    BOU   73240
6   BTRD    2965
7   BTRU    2990
8    CCB   26940
9    CCS   26629
10    DB   69421
11   DBB   22744
12    DT   28497
13   DTS   11382
14   PBB     880
15   PBS     882
16  SBRT   16388
17   SSD   87128
18  SSRT    6777
19   SSU   86691
20   TAG   14475
21   TAR   14117


### Load Lookback Periods
- This LOOKBACK_DF is what I use to re-test the previous 12-month period every 1-month
  1. So every month I look at the previous 12-months starting on the last day of the previous month
  2. I want to create new models from the previous 12-month period
  3. I will backtest/validate the models by running the current month data through the model

In [5]:
### Load Lookback Periods
#######################################
LOOKBACK_DF = Periods.lookback_df(LB_MONTHS=12,NEXT=False,MIN_DATE=None,MAX=None,LB_MIN_DATE=None)
LOOKBACK_DF = LOOKBACK_DF[['month_id','lookback_start','lookback_end']][(LOOKBACK_DF["month_id"] <= '2022_11')].head(12).reset_index(drop=True)
print(LOOKBACK_DF)

   month_id lookback_start lookback_end
0   2022_11     2021-11-01   2022-10-31
1   2022_10     2021-10-01   2022-09-30
2   2022_09     2021-09-01   2022-08-31
3   2022_08     2021-08-01   2022-07-31
4   2022_07     2021-07-01   2022-06-30
5   2022_06     2021-06-01   2022-05-31
6   2022_05     2021-05-01   2022-04-30
7   2022_04     2021-04-01   2022-03-31
8   2022_03     2021-03-01   2022-02-28
9   2022_02     2021-02-01   2022-01-31
10  2022_01     2021-01-01   2021-12-31
11  2021_12     2020-12-01   2021-11-30


## Model Creation Master Function
- This will obviously but a much longer and more involved function but the main takeways are:
   - Even though we need to re-create these models, I think it's save to build off of your analysis that Random Forest & All Features seems to work best, so let's limit the model creation to that general scope for now.
   - It should save the model information for the specific lookback period into that period's folder
   - It should save whatever information is neccesary (i.e. pickle file) so that in production mode it can make as fast of a prediction as possible.

In [6]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
import sklearn
import catboost as cb
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.decomposition import PCA
import pickle
import pathlib

#define your own mse and set greater_is_better=False
mse = make_scorer(mean_squared_error,greater_is_better=False)

from sklearn.model_selection import GridSearchCV


def create_model(model_creation_data,AGG,PCODE,MONTH_ID,
                 columns_dist, Scaling=None, n_test=0,
                 regression='rf'):
    print(f"\n\t{'-'*60}\n\tCREATE MODEL FOR '{PCODE}' | {MONTH_ID}\n\t{'-'*60}")
    ''' ADD FUNCTIONALITY HERE '''
    numeric_features = columns_dist['cols_attributes_basic']
    cat_features = columns_dist['cols_attributes_category']
    column_target = columns_dist['column_target']
    
    # Standard Scalar Transformer of numeric contineous data
    numeric_tr_SS = Pipeline(steps=[("scaler", StandardScaler())])
    
    # Minmax Transformer of numeric contineous data
    numeric_tr_MM = Pipeline(steps=[("scaler", MinMaxScaler())])
        
    # Catagegorcial transformers on catagorical attributes
    categorical_transformer = OneHotEncoder(handle_unknown="ignore")
    
    # Features and Target data
    X = model_creation_data[numeric_features+cat_features]
    y = model_creation_data[[column_target]]
    
    if Scaling:
        if Scaling == 'MinMax':
            numeric_transformer = numeric_tr_MM
        elif scaling=='StandardScalar':
            numeric_transformer = numeric_tr_SS
        
        preprocessor = ColumnTransformer(
            transformers=[
                ("num_transform", numeric_transformer, numeric_features),
                ("cat_transform", categorical_transformer, cat_features),
            ]
        )
    
    else:
        preprocessor = ColumnTransformer(
                transformers=[
                    ("cat_transform", categorical_transformer, cat_features),
                ],
                remainder='passthrough')
    
    pca = PCA()
    
    param = {
        'rf_regressor':{'regressor__n_estimators': [100, 150,
                                                       200, 250, 300,
                                                       350, 400, 450, 500, 550,
#                                                        600,700,800,900,1000
                                                      ],
                'regressor__max_features':['sqrt', 'log2'],
#                 'pca__n_components': [10, 15, 30, 45, 60, 75, 90, 100],
                                                       },
        'catboost':{'regressor__iterations': [100, 150, 200],
                    'regressor__learning_rate': [0.1, 0.2, 0.3, 0.4],
                    'regressor__depth': [4, 6, 8, 10],
                    'regressor__l2_leaf_reg': [0.1, 0.2, 0.5, 1, 1.5],
                    'regressor__verbose': [True]
#                 'pca__n_components': [10, 15, 30, 45, 60, 75, 90, 100],                   
                   }
               
               }
    if regression == 'rf':
        rfRegressor = RandomForestRegressor(random_state=42)
        param_grid = param['rf_regressor']
        pipe = Pipeline(
            steps=[("preprocessor", preprocessor), ("regressor", rfRegressor)])
    elif regression=='catboost':
        catboost = cb.CatBoostRegressor(loss_function='RMSE', task_type="GPU")
        param_grid = param['catboost']
        pipe = Pipeline(
            steps=[("preprocessor", preprocessor), ("regressor", catboost)])
    elif regression=='DT':
        None
        #Add code
        

    search = GridSearchCV(estimator = pipe,
                          param_grid=param_grid,
                          return_train_score=True,
                          scoring=mse,
#                           verbose=2,
                          refit = True,
                          n_jobs=-1)
    
    print(f"\t1. Train Random Forest Model using All Features")
    search.fit(X, y.values.flatten())
    best_fit = search.best_estimator_

    MODEL_CREATION_DIRECTORY = f'Sources/aggs/{AGG}/Results/Pattern_Models/{PCODE}/test_{n_test}/{MONTH_ID}/'
    pathlib.Path(os.path.join(WD, MODEL_CREATION_DIRECTORY)).mkdir(parents=True, exist_ok=True)    
    
    print(f"\t2. Create Model Directory Folder\n\t     {MODEL_CREATION_DIRECTORY}")
    print(f"\t3. Save Neccesary Files for Fast Predictions in Production")
    pickle.dump(best_fit, open(os.path.join(WD, MODEL_CREATION_DIRECTORY) + 'model.sav', 'wb'))  
    print(f"\t{'-'*60}\n")
    return search
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [25]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import seaborn as sns
import pickle
import matplotlib
import matplotlib.pyplot as plt

def analyse_model(model_creation_data, model_production_data,AGG,PCODE,MONTH_ID, n_test=0):
    print(f"\n\t{'-'*60}\n\tANALYSE MODEL FOR '{PCODE}' | {MONTH_ID}\n\t{'-'*60}")
    
    numeric_features = columns_dist['cols_attributes_basic']
    cat_features = columns_dist['cols_attributes_category']
    column_target = columns_dist['column_target']
    X_creation_data = model_creation_data[numeric_features+cat_features]
    y_creation_data = model_creation_data[[column_target]]
    
    X_production_data = model_production_data[numeric_features+cat_features]
    y_production_data = model_production_data[[column_target]]
    
    MODEL_CREATION_DIRECTORY = f'Sources/aggs/{AGG}/Results/Pattern_Models/{PCODE}/test_{n_test}/{MONTH_ID}/'
    MODEL_PATH = os.path.join(WD, MODEL_CREATION_DIRECTORY, 'model.sav')
    
    print(f"\t1. Load Saved Model Files created from 'model_creation_data'")
    model = pickle.load(open(MODEL_PATH, 'rb'))
    
    print(f"\t2. Running Inference on from 'model_creation_data'")
    prediction_creation_data =  model.predict(X_creation_data)
    model_creation_data['predict'] = prediction_creation_data
    model_creation_data.insert(1, 'predict', model_creation_data.pop('predict'))
    
    print(f"\t3. Running Inference on from 'model_creation_data'")
    prediction_production_data =  model.predict(X_production_data)
    model_production_data['predict'] = prediction_production_data
    model_production_data.insert(1, 'predict', model_production_data.pop('predict'))
    
    mse_cr = mean_squared_error(y_creation_data['_TARGET'].values.flatten(), prediction_creation_data)
    mse_pr = mean_squared_error(y_production_data['_TARGET'].values.flatten(), prediction_production_data)
    print(f"\t4. MSE on model_production_data:{mse_pr}")
    
    corr_cr = pearsonr(y_creation_data['_TARGET'].values.flatten(), prediction_creation_data).statistic
    corr_pr = pearsonr(y_production_data['_TARGET'].values.flatten(), prediction_production_data).statistic
    print(f"\t4. MSE on model_production_data:{corr_pr}")
    
    result = {'PCODE': PCODE, 'MONTH_ID': MONTH_ID,
              'Corr_cr': corr_cr, 'Corr_pr': corr_pr,
             'MSE_cr': mse_cr, 'MSE_pr': mse_pr}

    
    plt.figure()
    plt.tight_layout()
    sns.set(rc={"figure.figsize":(10, 5)})
    sns.regplot(x="_TARGET", y="predict", data=model_production_data);
    plt.show()
    
    ## Plot by Date
    plt.figure()
    sns.set(rc={"figure.figsize":(10, 5)})
    sns.lineplot(x = "_UTILITY_date", y = "_TARGET",data = model_creation_data)
    sns.lineplot(x = "_UTILITY_date", y = "_TARGET",data = model_production_data)
    sns.lineplot(x = "_UTILITY_date", y = "predict",data = model_creation_data)
    sns.lineplot(x = "_UTILITY_date", y = "predict",data = model_production_data)
    plt.legend(labels=['creation_data_target', 'creation_data_predict',
                       'production_data_target', 'production_data_predict',])

    plt.show()

    return result
    
    
    
    
    
#     ''' ADD FUNCTIONALITY HERE '''
    
#     print(f"\t2. Create Prediction Column in 'model_production_data'")
    

### Loop through Lookback Periods
- Below is an example of what I would like to be able to do in creating & testing models for each lookback period


In [1]:
ls = []
for index, row in LOOKBACK_DF[0:1].iterrows(): # <-- Set to just 1 loop for this example, but I will run this for all periods
    
    MONTH_ID = row["month_id"]
    lookback_start = pd.to_datetime(row["lookback_start"]).strftime('%Y-%m-%d')
    lookback_end = pd.to_datetime(row["lookback_end"]).strftime('%Y-%m-%d')
    LAST_MO_ID = pd.to_datetime(row["lookback_end"]).strftime('%Y_%m')
    print(f"\n{'='*100}\nRun for {MONTH_ID} \t Lookback Range {lookback_start} to {lookback_end}\n{'='*100}\n")
    
    for PCODE in ALL_CODES: # <-- Set to just 1 PCODE for this example, but I will run this for all PCODES
        
        ### Load Production Data (For Backtesting)
        #######################################
        model_production_data = Drct.load_pcode(AGG=AGG,PCODE=PCODE,PP=True,LAST_MO_ID=MONTH_ID,MONTHS_BACK=1,PRINT=True)
        ### Model Creation Data
        #######################################
        model_creation_data = Drct.load_pcode(AGG=AGG,PCODE=PCODE,PP=True,LAST_MO_ID=LAST_MO_ID,MONTHS_BACK=12,PRINT=True)
        model_creation_data['_UTILITY_date'] = pd.to_datetime(model_creation_data['_UTILITY_date'], format='%Y/%m/%d')
        model_production_data['_UTILITY_date'] = pd.to_datetime(model_production_data['_UTILITY_date'], format='%Y/%m/%d')

        model_production_data['_ftN_time'] = model_production_data['_UTILITY_puid'].apply(lambda x: int(x.split('_')[1]))
        model_creation_data['_ftN_time'] = model_creation_data['_UTILITY_puid'].apply(lambda x: int(x.split('_')[1]))  
        
        
        
        
# #         ### Define Column Types
# #         #######################################
#         cols = model_creation_data.columns.tolist()
#         basic_is = lambda x: True in [ i in x for i in ['_ftN_',]]
#         category_is = lambda x: True in [ i in x for i in ['_ftC_',]]
#         utility_is = lambda x: True in [ i in x for i in ['_UTILITY_',]]
#         cols_attributes_basic = [ i for i in cols if basic_is(i) ]
#         cols_attributes_category = [ i for i in cols if category_is(i) ]
#         cols_utility = [ i for i in cols if utility_is(i) ]
#         column_target = '_TARGET'
        
#         columns_dist = {'cols_attributes_basic': cols_attributes_basic,
#                 'cols_attributes_category': cols_attributes_category,
#                 'column_target': column_target}
        
        
        ## RUN :: Model Creation
        ######################################
        
#         create_model(model_creation_data,AGG,PCODE,MONTH_ID, columns_dist, n_test=0,regression='rf')
        
        ### RUN :: Model Production Predict
        #######################################
#         model_production_data, stats = run_production_predict(model_production_data,AGG,PCODE,MONTH_ID)
        
        ### RUN :: Model Analysis
        #######################################
        result = analyse_model(model_creation_data, model_production_data,AGG,PCODE,MONTH_ID, n_test=0)
#         ls.append(result)

        



NameError: name 'LOOKBACK_DF' is not defined

In [58]:
result

{'PCODE': '3BPD',
 'MONTH_ID': '2022_11',
 'Corr_cr': 0.9923488759079054,
 'Corr_pr': 0.11419990133524326,
 'MSE_cr': 5.891390299121038e-06,
 'MSE_pr': 5.366328144919452e-05}

### NOTES About Column Naming Structure
- There are 3 primary types of columns:
    1. The target column (y) for creating/validating predictions. This column is named '_TARGET'
    2. Utility columns that is just here to help me inspect the data but these should not be used in creating predictions (starts with '_UTILITY_')
    2. Basic Attribute columns (X) that are numerical (but not neccesarily all linier) used for creating predictions (starts with '_ftN_')
    2. Categorical Attribute columns (X) that have already been converted from different strings to intigers used for creating predictions (starts with '_ftC_')


In [26]:
print(f"\nUtility Column Examples \t {cols_utility[0:5]}")
print(f"Basic Attributes Examples \t {cols_attributes_basic[0:5]}")
print(f"Category Attributes Examples \t {cols_attributes_category[0:5]}")
print(f"Target Column Is \t\t {column_target}\n")



Utility Column Examples 	 ['_UTILITY_date', '_UTILITY_puid']
Basic Attributes Examples 	 ['_ftN_open', '_ftN_high', '_ftN_low', '_ftN_close', '_ftN_volume']
Category Attributes Examples 	 ['_ftC_gap_type', '_ftC_gap_level', '_ftC_gap_code']
Target Column Is 		 _TARGET



### NOTES About model_production_data & model_creation_data
- The function Drct.load_pcode() Loads the model data for a specific date range.
- Notice the date range for the 'model_production_data' is a 1-month period
- Notice the date range for the 'model_creation_data' is the previous 12-month period


## Model Production Master Function
- This function is meant to be a "mock version" of what I will end up using with live real-time data.
- In live production mode, I will take real-time data (just like 'model_production_data' except only a few rows at a time and not a whole month) and send it to this function in order to get a new 'predict' column in return.
- For backtesting purposes I would also like to have a 'stats' dataframe returned as well with run-time and accuracy scores.


In [ ]:
model_production_data

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def run_production_predict(model_production_data,AGG,PCODE,MONTH_ID):
    
    stats_start_time = datetime.now()
    
    MODEL_CREATION_DIRECTORY = f'Sources/aggs/{AGG}/Results/Pattern_Models/{PCODE}/{MONTH_ID}/'
    print(f"\n\t{'-'*60}\n\tRUN PRODUCTION PREDICT FOR '{PCODE}' | {MONTH_ID}\n\t{'-'*60}")
    
    print(f"\t1. Load Saved Model Files created from 'model_creation_data'")
    
    
    
    print(f"\t2. Create Prediction Column in 'model_production_data'")
    
    ''' ADD FUNCTIONALITY HERE '''
    model_production_data['predict'] = np.nan # <-- Add prediction here
    model_production_data.insert(1, 'predict', model_production_data.pop('predict'))
    
    print(f"\t3. Create Stats KPI CSV File with 'model_production_data' results")
    
    ''' ADD FUNCTIONALITY HERE '''
    
    stats_run_seconds = (datetime.now() - stats_start_time).total_seconds()
    raw = {
    'pcode': [PCODE],
    'month_id': [MONTH_ID],
    'prediction_accuracy': [None], # <-- Something that compares '_TARGET' to 'predict'
    'prediction_seconds': [stats_run_seconds], # <-- Something similar to this 
    }
    stats = pd.DataFrame(data=raw)
    
    print(f"\t4. Save the updated 'model_production_data' and 'stats' to CSV")
    
    print(f"\t5. Return the updated 'model_production_data' and 'stats' from function")
    
    print(f"\t{'-'*60}\n")
    
    return model_production_data, stats
          
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [1]:
### RUN :: Model Production Predict
#######################################
model_production_data, stats = run_production_predict(model_production_data,AGG,PCODE,MONTH_ID)


NameError: name 'run_production_predict' is not defined

In [ ]:
print(stats)

In [ ]:
model_production_data

In [ ]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())